&nbsp;

# 19 - Théorie des modèles SciML

---

&nbsp;

## 1. Introduction

&nbsp;

> #### De quoi parle-t-on quand on parle d'un système *dynamique* ?

Un système sera dit dynamique si sont état évolue selon une règle d'évolution dans le temps (et pas juste des corrélations). La "dynamique" parle de ce qui va arriver, pas seulement de ce qui est observé. Les lois physiques sont causales, locales dans le temps et invariantes, ainsi on cherche pas :

$$
P(X_{t+1} | X_t)
$$

Mais plutôt :

$$
\frac{dX}{dt} = F(X)
$$

Mathématiquement, la caractéristique de *dynamisme* s'attribue à la condition de 4 propriétés clés :

- une variable d'état : $X(t) \in \mathbb{R}^n$ (l'état contient toute les informations nécessaires pour "prédire" le futur)
- une loi d'évolution : $\frac{dX}{dt} = F(X) \space \text{avec} \space F:\mathbb{R}^n \to \mathbb{R}^n$ (où $F$ est la dynamique en question)
- une localité temporelle (causalité) : $\dot{X}(t) = F(X(t))$ (le futur dépend du présent, pas du futur)
- une invariance dans le temps : $F_t = F$ (c'est une loi)

&nbsp;

Dans la mesure où l'on souhaite, plus basiquement, prédire les valeurs futures de SST dans la zone Manche. Il y a deux approches fondamentalement différentes pour prédire :

- une prédiction discrète, on apprend : $X(t + \Delta t) = G(X(t))$ (on apprend une trajectoire, c'est ce que font les régressions classiques, les RNN, les modèles autorégressifs, etc.)
- une prédiction continue, on apprend : $\frac{dX}{dt} = F(X)$

Sans creuser plus dans le détail du raisonnement mathématique, l'approche continue est plus informative, ce qui au final fournit des prédictions plus robustes. C'est pour cela qu'on cherche à modéliser via SciML pour capturer cette dynamique, les phénomènes physiques de la sorte sont le plus souvent caractérisable de systèmes "dynamiques".

&nbsp;

---

&nbsp;

## 2. Panorama et choix des modèles SciML

Commençons par énoncer que le choix d'un modèle SciML n'est pas indépendant de la représentation d'état. C'est-à-dire que nos 2 états réduits possèdent chacuns des caractéristiques encourageant l'utilisation d'un modèle plus qu'un autre. On ne modèlise pas la même dynamique dans les deux cas.

&nbsp;

> #### Quelles sont les grandes familles de modèles SciML ?

On l'a énoncé plusieurs fois, une approche continue implique le principe suivant :

$$
\frac{dX}{dt} = F(X)
$$

Chacune des familles se distinguent par la manière dont elles représentent $F$ et par les hypothèses qu'elles imposent.

#### **A. Dynamique symbolique**

Ici, on cherche une équation :

$$
F(X) = \underset{k}{\sum} \xi_k \phi_k(X)
$$

Cela implique une structure explicite, de forme fermée avec une interprétation physique assez directe. On parle des méthodes dites *sparse regression* (SINDy par exemple).

#### **B. Dynamique paramétrique universelle**

Ici, on approxime $F$ sans hypothèse de forme.

$$
F(X)\approx f_\theta(X)
$$

Cela se présente de la forme d'un réseau de neurone ayant plusieurs avantages : flexibilité, peu d'apriori, boîte noire. On parle des méthodes Neural ODE/PDE.

#### **C. Dynamique contrainte par la physique**

Ici, on pose l'hypothèse que l'on connaît